# step5, 역전파 이론

# Step6, 수동 역전파

## 6.1 Variable 클래스 추가 구현

역전파에 대응하는 Variable 클래스를 구현  
그러기 위해 통상값(data)과 더불어 그에 대응하는 미분값(grad)도 저장하도록 확장

인스턴스 변수인 data와 grad는 모두 넘파이의 다차원 배열(ndarray)이라고 가정  
grad는 None으로 초기화해둔 다음, 나중에 실제로 역전파를 하면 미분값을 계산하여 대입

In [2]:
import numpy as np

In [3]:
class Variable:
    def __init__(self, data):
        self.data = data 
        self.grad = None 

## 6.2 Function 클래스 추가 구현

이전 단계까지의 Function 클래스는 일반적인 계산을 하는 순전파(forward 메서드) 기능만 지원하는 상태 
- 미분을 계산하는 역전파(backward 메서드)
- forward 메서드 호출시 건네받은 Variable 인스턴스 유지

In [4]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input  # 입력변수를 기억(보관)한다. 
        return output
    
    def forward(self, x):
        raise NotImplementedError() 

    def backward(self, gy):
        raise NotImplementedError()

## 6.3 Square와 Exp 클래스 추가 구현

이어서 Function을 상속한 구체적인 함수에서 역전파(backward)를 구현  

In [5]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y 
    
    def backward(self, gy):
        x = self.input.data 
        gx = 2 * x * gy 
        return gx

In [6]:
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

## 6.4 역전파 구현 


In [7]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

In [8]:
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


# Step7, 역전파 자동화

이전 단계에서 역전파를 동작시켰다. 그러나 역전파 계산 코드를 수동으로 조합했다.  
새로운 계산을 할 때마다 역전파 코드를 직접 작성해야 한다.

이제부터 역전파를 자동화  
계산(순전파)을 한 번만 해주면 어떤 계산이라도 상관없이 역전파가 자동으로 이루어지는 구조

**Define-by-Run**  
Define-by-Run : 딥러닝에서 수행하는 계산들을 계산 시점에 '연결'하는 방식  
'동적 계산 그래프'라고더 함.

함수의 순서를 리스트 형태로 저장해두면 나중에 거꾸로 추적하는 식으로 역전파를 자동화


## 7.1 역전파 자동화의 시작

변수와 함수의 관계  
변수 입장에서 변수를 만들어준 함수는 창조자  
창조자인 함수가 존재하지 않는 변수는 사용자에 의해 만들어진 변수로 간주 

일반적인 계산(순전파)이 이루어지는 시점에 '관계'를 맺어주도록 만듬(함수와 변수를 연결)

creator라는 인스턴스 변수 추가

In [9]:
import numpy as np


class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

In [10]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)  # Set parent(function)
        self.input = input
        self.output = output  # Set output
        return output

순전파를 계산하면 그 결과 output이라는 Variable 인스턴스가 생성  
이때 생성된 output에 '내가 너의 창조자임'을 기억시킨다.  
이 부분이 '연결'을 동적으로 만드는 기법의 핵심  
그런 다음 앞으로를 위해 output을 인스턴스 변수에 저장

In [11]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx


class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [12]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

assert y.creator == C 
assert y.creator.input == b 
assert y.creator.input.creator == B 
assert y.creator.input.creator.input == a 
assert y.creator.input.creator.input.creator == A 
assert y.creator.input.creator.input.creator.input == x

assert : '단언하다', '단호하게 주장하다'  
assert '주장' 

'주장' 그 평가 결과가 True가 아니면 예외가 발생  
assert 문은 조건을 충족하는지  여부를 확인하는데 사용

**Define-by-Run**  
데이터를 흘려보냄으로써(Run함으로써) 연결이 규정된다는(Define된다는) 뜻

노드들의 연결로 이루어진 데이터 구조를 "링크드 리스트(linked list)**  
노드는 그래프를 구성하는 요소이며, link는 다른 노드를 가리키는 참조를 뜻함  
linked list라는 데이터 구조를 이용해 계산 그래프를 표현한것이다.

## 7.2 역전파 도전!

y에서 b까지의 역전파를 시도 (그림을 보면서)

흐름  
1. 함수를 가져온다.  
2. 함수의 입력을 가져온다.  
3. 함수의 backward 메서드를 호출한다. 

In [13]:
y.grad = np.array(1.0)

C = y.creator   # 1. 함수를 가져온다.
b = C.input     # 2. 함수의 입력을 가져온다. 
b.grad = C.backward(y.grad) # 3. 함수의 backward 메서드를 호출한다

In [14]:
B = b.creator   # 1. 함수를 가져온다.
a = B.input     # 2. 함수의 입력을 가져온다. 
a.grad = B.backward(b.grad) # 3. 함수의 backward 메서드를 호출한다.

In [15]:
A = a.creator   # 1. 함수를 가져온다.
x = A.input     # 2. 함수의 입력을 가져온다. 
x.grad = A.backward(a.grad) # 3. 함수의 backward 메서드를 호출한다.

print(x.grad)

3.297442541400256


## 7.3 backward 메서드 추가 

In [25]:
class Variable:
    def __init__(self, data):
        self.data = data 
        self.grad = None
        self.creator = None 

    def set_creator(self, func):
        self.creator = func 

    def backward(self):
        f = self.creator    # 1. 함수를 가져온다. 
        if f is not None:
            x = f.input     # 2. 함수의 입력을 가져온다.
            x.grad = f.backward(self.grad)
            x.backward()    # 하나 앞 변수의 backward 메서드를 호출한다(재귀)

**Note_** Variable 인스턴스의 creator가 None이면 역전파가 중단. 창조자가 없으므로 이 Variable인스턴스는 함수 바깥에서 생성됐음을 뜻함

In [29]:
# 함수 정의
A = Square() 
B = Exp() 
C = Square() 

# 데이터 정의와 순전파
x = Variable(np.array(0.5))
a = A(x)
print(a.data)
b = B(a)
print(b.data)
y = C(b)
print(y.data)

# 역전파 
y.grad = np.array(1.0)
y.backward()
print(x.grad)

0.25
1.2840254166877414
1.648721270700128
3.297442541400256
